In [ ]:
"""Plot MED OFF vs. ON TRGC (Figure 3C)."""

import os
import sys
from pathlib import Path
import numpy as np
import copy
import pandas as pd
import pte_stats
import pte_decode
from matplotlib import pyplot as plt
from scipy.stats import sem

cd_path = Path(os.getcwd()).absolute().parent
sys.path.append(os.path.join(cd_path, "coherence"))

import matplotlib
matplotlib.rc('xtick', labelsize=6)
matplotlib.rc('ytick', labelsize=6)
matplotlib.rc('legend', fontsize=6)
matplotlib.rc("font", size=6, family="Arial")
matplotlib.rc('axes', labelsize=7)
matplotlib.rc('axes', titlesize=7)
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

prop_cycle = plt.rcParams["axes.prop_cycle"]
def_colors = prop_cycle.by_key()["color"]

n_perm = 100_000
two_sided = True
alpha = 0.05

FOLDERPATH_ANALYSIS = "Path_to\\Project\\Analysis"
FOLDERPATH_FIGURES = os.path.join(os.path.dirname(os.getcwd()), "figures")

lfreq = 4.0
hfreq = 40.0

fband_func = np.mean

subregions_mapping = {
    "Motor": "motor",
    "Sensory": "sensory",
}

fbands = {
    "alpha": (8, 12),
    "low_beta": (12, 20),
    "high_beta": (20, 30)
}

In [ ]:
# MOTOR CORTEX -> STN TRGC (Figure 3C)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, f"task-Rest_acq-multi_run-multi_con_granger_regional-MedOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["frequencies"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["frequencies"] = [freqs] * len(data["seed_types"])
data = pd.DataFrame.from_dict(data)

trgc_off = np.array(data["connectivity-trgc"][(data["med"] == "Off") & (data["seed_subregions"] == "motor")].to_list())
trgc_on = np.array(data["connectivity-trgc"][(data["med"] == "On") & (data["seed_subregions"] == "motor")].to_list())

np.random.seed(44)

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]

trgc_off = trgc_off[:, lfreq_i:hfreq_i+1]
trgc_on = trgc_on[:, lfreq_i:hfreq_i+1]

trgc_off_sem = sem(trgc_off)
trgc_on_sem = sem(trgc_on)

fig, axis = pte_decode.lineplot_prediction_compare(
    x_1=trgc_off.T,
    x_2=trgc_on.T,
    times=np.arange(lfreq, hfreq+0.5, 0.5),
    data_labels=["OFF therapy", "ON levodopa"],
    x_label="Frequency (Hz)",
    y_label="Time-reversed\nGranger causality (A.U.)",
    two_tailed=two_sided,
    paired_x1x2=True,
    n_perm=n_perm,
    correction_method="cluster_pvals",
    title="Motor cortex -> STN",
)

for fband_name, fband_lims in fbands.items():
    stat, pval = pte_stats.permutation_onesample(
        data_a=fband_func(trgc_off[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        data_b=fband_func(trgc_on[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        n_perm=n_perm, two_tailed=two_sided
    )
    if pval < alpha:
        axis.axvspan(fband_lims[0], fband_lims[1], color=[0.8, 0.8, 0.8], alpha=0.3, linewidth=0)
        prefix = "[SIGNIFICANT]"
    else:
        prefix = ""
    print(f"{prefix} {fband_name}: stat={stat}; p-val={pval}")

axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.legend(handlelength=0.5, handletextpad=0.5, labelspacing=0, borderaxespad=0, loc="upper left")
leg = axis.get_legend()
leg.legend_handles[0].set_linewidth(1.5)
leg.legend_handles[1].set_linewidth(1.5)
leg.set_frame_on(False)
axis.set_xticks(np.arange(5, 45, 5))
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.xaxis.labelpad = 0
axis.yaxis.labelpad = 1
axis.set_yticks([0, 0.05, 0.1])

fig.set_size_inches(1.8, 1.35)

fig.savefig(os.path.join(FOLDERPATH_FIGURES, "Manuscript_TRGC_Med_motor_cortex-STN.pdf"))

In [ ]:
# SENSORY CORTEX -> STN TRGC (Figure S2)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, f"task-Rest_acq-multi_run-multi_con_granger_regional-MedOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["frequencies"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["frequencies"] = [freqs] * len(data["seed_types"])
data = pd.DataFrame.from_dict(data)

trgc_off = np.array(data["connectivity-trgc"][(data["med"] == "Off") & (data["seed_subregions"] == "sensory")].to_list())
trgc_on = np.array(data["connectivity-trgc"][(data["med"] == "On") & (data["seed_subregions"] == "sensory")].to_list())

np.random.seed(44)

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]

trgc_off = trgc_off[:, lfreq_i:hfreq_i+1]
trgc_on = trgc_on[:, lfreq_i:hfreq_i+1]

trgc_off_sem = sem(trgc_off)
trgc_on_sem = sem(trgc_on)

fig, axis = pte_decode.lineplot_prediction_compare(
    x_1=trgc_off.T,
    x_2=trgc_on.T,
    times=np.arange(lfreq, hfreq+0.5, 0.5),
    data_labels=["OFF therapy", "ON levodopa"],
    x_label="Frequency (Hz)",
    y_label="Time-reversed\nGranger causality (A.U.)",
    two_tailed=two_sided,
    paired_x1x2=True,
    n_perm=n_perm,
    correction_method="cluster_pvals",
    title="Sensory cortex -> STN",
)

for fband_name, fband_lims in fbands.items():
    stat, pval = pte_stats.permutation_onesample(
        data_a=fband_func(trgc_off[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        data_b=fband_func(trgc_on[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        n_perm=n_perm, two_tailed=two_sided
    )
    if pval < alpha:
        axis.axvspan(fband_lims[0], fband_lims[1], color=[0.8, 0.8, 0.8], alpha=0.3, linewidth=0)
        prefix = "[SIGNIFICANT]"
    else:
        prefix = ""
    print(f"{prefix} {fband_name}: stat={stat}; p-val={pval}")

axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.legend(handlelength=0.5, handletextpad=0.5, labelspacing=0, borderaxespad=0, loc="upper left")
leg = axis.get_legend()
leg.legend_handles[0].set_linewidth(1.5)
leg.legend_handles[1].set_linewidth(1.5)
leg.set_frame_on(False)
axis.set_xticks(np.arange(5, 45, 5))
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.xaxis.labelpad = 0
axis.yaxis.labelpad = 1
axis.set_yticks([0, 0.04, 0.08])

fig.set_size_inches(1.8, 1.35)

fig.savefig(os.path.join(FOLDERPATH_FIGURES, "Manuscript_TRGC_Med_sensory_cortex-STN.pdf"))

In [ ]:
# Find significant periods of connectivity (whole cortex -> STN; OFF)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, f"task-Rest_acq-multi_run-multi_con_granger_whole-MedOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["frequencies"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["frequencies"] = [freqs] * len(data["seed_types"])
data = pd.DataFrame.from_dict(data)

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(100)
plot_freqs = freqs[lfreq_i:hfreq_i+1]

trgc_off = np.array(data["connectivity-net_trgc"][data["med"] == "Off"].to_list())
trgc_on = np.array(data["connectivity-net_trgc"][data["med"] == "On"].to_list())
trgc = np.mean([trgc_off, trgc_on], axis=0)
trgc = trgc[:, lfreq_i:hfreq_i+1]

fig, axis = pte_decode.lineplot_prediction_compare(
    x_1=trgc.T,
    x_2=np.zeros_like(trgc).T,
    times=np.array(plot_freqs),
    data_labels=["TRGC", "Baseline"],
    x_label="Frequency (Hz)",
    y_label="TRGC (A.U.)",
    two_tailed=two_sided,
    paired_x1x2=True,
    n_perm=n_perm,
    correction_method="cluster_pvals",
    title="Cortex -> STN (OFF)",
    colour=[def_colors[0], "k"]
)

In [ ]:
# Find significant periods of connectivity (motor cortex -> STN; OFF)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, f"task-Rest_acq-multi_run-multi_con_granger_regional-MedOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["frequencies"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["frequencies"] = [freqs] * len(data["seed_types"])
data = pd.DataFrame.from_dict(data)

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]

trgc_off = np.array(data["connectivity-trgc"][(data["med"] == "Off") & (data["seed_subregions"] == "motor")].to_list())
trgc_off = trgc_off[:, lfreq_i:hfreq_i+1]

fig, axis = pte_decode.lineplot_prediction_compare(
    x_1=trgc_off.T,
    x_2=np.zeros_like(trgc_off).T,
    times=np.array(plot_freqs),
    data_labels=["TRGC", "Baseline"],
    x_label="Frequency (Hz)",
    y_label="TRGC (A.U.)",
    two_tailed=two_sided,
    paired_x1x2=True,
    n_perm=n_perm,
    correction_method="cluster_pvals",
    title="Cortex -> STN (OFF)",
    colour=[def_colors[0], "k"]
)